In [1]:
from transformers import DistilBertForSequenceClassification,DistilBertTokenizer
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, TensorDataset,DataLoader
from sklearn.model_selection import train_test_split
import torch

I0306 00:42:27.453490 4617063936 file_utils.py:41] PyTorch version 1.4.0 available.


In [2]:
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=1)
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')


# Load saved best BERT model
model.load_state_dict(torch.load(PATH))
model.eval()

I0306 00:42:32.385972 4617063936 configuration_utils.py:256] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/distilbert-base-uncased-config.json from cache at /Users/Bart/.cache/torch/transformers/a41e817d5c0743e29e86ff85edc8c257e61bc8d88e4271bb1b243b6e7614c633.8949e27aafafa845a18d98a0e3a88bc2d248bbc32a1b75947366664658f23b1c
I0306 00:42:32.387161 4617063936 configuration_utils.py:292] Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": null,
  "dim": 768,
  "do_sample": false,
  "dropout": 0.1,
  "eos_token_ids": null,
  "finetuning_task": null,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1"
  },
  "initializer_range": 0.02,
  "is_decoder": false,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1
  },
  "length_penalty": 1.0,
  "max_length": 20,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 1

In [4]:
tokenized_utterances = train_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))
tokenized_utterances_test = test_df.text.apply(lambda x: tokenizer.encode(x,add_special_tokens=True))

In [5]:
# pad sentences for training set
max_len = max(map(len,tokenized_utterances))
padded_utterances = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_utterances])
attention_masked_utterances = np.where(padded_utterances != 0,1,0)

In [6]:
# pad sentences for test set
padded_utterances_test = np.array([ i+[0]*(max_len-len(i))  for i in tokenized_utterances_test])
attention_masked_utterances_test = np.where(padded_utterances_test != 0,1,0)

In [7]:
# Create tensors

X_train = torch.tensor(padded_utterances)
X_train_attention = torch.tensor(attention_masked_utterances)
y_train = torch.tensor(np.array(train_df.sentiment_score.values)[:,np.newaxis], dtype=torch.float32)

X_test = torch.tensor(padded_utterances_test)
X_test_attention = torch.tensor(attention_masked_utterances_test)
y_test = torch.tensor(np.array(test_df.sentiment_score.values)[:,np.newaxis], dtype=torch.float32)


train_data = TensorDataset(X_train, X_train_attention, y_train)
train_loader = DataLoader(train_data,batch_size=16, shuffle=True)

test_dataset = TensorDataset(X_test, X_test_attention)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [8]:
# Train model

NUM_EPOCHS = 1
LEARNING_RATE = 0.01
optimizer =torch.optim.SGD(model.parameters(), lr=LEARNING_RATE)
loss_fn = torch.nn.BCEWithLogitsLoss()
for i in range(NUM_EPOCHS):
    model.train()
    for X_batch,X_attention_batch,y_batch in train_loader:
        output = model(X_batch,attention_mask=X_attention_batch,labels=None)
        y_pred = output[0]
        loss = loss_fn(y_pred,y_batch)
        print(loss)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

tensor(0.6954, grad_fn=<BinaryCrossEntropyWithLogitsBackward>)


KeyboardInterrupt: 